# Setup

In [1]:
import torch.nn as nn
import torch
import numpy as np
from helpers import Item, output_to_file,random_ndarray,random_tensor,build_sam_test,input_to_file


# Loading image

In [2]:
import cv2
image = cv2.imread('images/truck.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
print(image.shape)
items =[Item("image",image,"TensorInt")]
output_to_file("image",items)

(1200, 1800, 3)


## Common
#### LayerNorm2d

In [ ]:
from segment_anything.modeling.common import LayerNorm2d

layer_norm = LayerNorm2d(256,0.1)

# Forward
input = random_tensor([2,256,16,16],1)
output = layer_norm(input)
items = [Item("input", input, "TensorFloat"), Item("output", output, "TensorFloat")]
output_to_file("layer_norm_2d",items)
del layer_norm, input, output,items

#### MLPBlock

In [ ]:
from segment_anything.modeling.common import MLPBlock
mlp_block = MLPBlock(256,256,nn.GELU)
input_to_file("mlp_block",mlp_block)

# Forward
input = random_tensor([256,256],5)
output = mlp_block(input)
items = [Item("input", input, "TensorFloat"), Item("output", output, "TensorFloat")]
output_to_file("mlp_block",items)
del mlp_block, input, output,items

# Image encoder

#### PatchEmbeded

In [ ]:
from segment_anything.modeling.image_encoder import PatchEmbed

patch_embed = PatchEmbed((16,16),(16,16),(0,0),3,320)
input_to_file("patch_embed",patch_embed)

# Forward
input = random_tensor([1,3,512,512],3)
output = patch_embed(input)
items = [Item("input", input, "TensorFloat"), Item("output", output, "TensorFloat")]
output_to_file("patch_embed",items)
del patch_embed, input, output,items

#### Attention

In [ ]:
from segment_anything.modeling.image_encoder import get_rel_pos,add_decomposed_rel_pos

# Get rel pos
q_size = 32
k_size = 32
input = random_tensor([127,40],1)
output = get_rel_pos( q_size, k_size, input)
items = [Item("input",input,"TensorFloat"),Item("output", output, "TensorFloat")]
output_to_file("get_rel_pos",items)
del input, output

In [ ]:
# Add decomposed rel pos
attn = random_tensor([200,49,49],2)
q = random_tensor([200,49,20],3)
relo_pos_h = random_tensor([20,20],4)
relo_pos_w = random_tensor([20,20],5)
q_size = (7,7)
k_size = (7,7)
output = add_decomposed_rel_pos(attn,q,relo_pos_h,relo_pos_w,q_size,k_size)
items = [Item("attn", attn, "TensorFloat"), Item("q", q, "TensorFloat"), Item("q_size", q_size, "Size"), Item("k_size", k_size, "Size"), Item("output", output, "TensorFloat")]
output_to_file("add_decomposed_rel_pos",items)
del attn,q,relo_pos_h,relo_pos_w,q_size,k_size,output,items

In [ ]:
from segment_anything.modeling.image_encoder import Attention

# Attention
attention = Attention(320, 16 ,True ,True ,True, (14, 14))
input_to_file("attention",attention)

# Forward
input = random_tensor([25,14,14,320],1)
output = attention(input)
items = [Item("input", input, "TensorFloat"), Item("output", output, "TensorFloat")]
output_to_file("attention",items)
del input
del output
del attention

In [ ]:
from segment_anything.modeling.image_encoder import  window_partition,window_unpartition

# Window partition
input = random_tensor([2,256,16,16],1)
output,size = window_partition(input,16)
items = [Item("input", input, "TensorFloat"), Item("output", output, "TensorFloat"), Item("size", size, "Size")]
output_to_file("window_partition",items)

# Window unpartition
input = random_tensor([2,256,16,16],2)
output = window_unpartition(input,16,(16,16),(14,14))
items = [Item("input", input, "TensorFloat"), Item("output", output, "TensorFloat")]
output_to_file("window_unpartition",items)
del input, output, items

#### Block

In [ ]:
from segment_anything.modeling.image_encoder import Block

#Block
block = Block(80,8,4.0,True,nn.LayerNorm,nn.GELU,True,True,14,(16,16))
input_to_file("block",block)
#Forward
input = random_tensor([1,16,16,80],1)
output = block(input)
items = [Item("input", input, "TensorFloat"), Item("output", output, "TensorFloat")]
output_to_file("block",items)
del block, input, output

#### Image encoderViT

In [ ]:
from segment_anything.modeling.image_encoder import ImageEncoderViT

image_encoder = ImageEncoderViT(4,4,3,80,4,16,4.0,32,True,nn.LayerNorm,nn.GELU,True,True,True,14,[7,15,23,31])
input_to_file("image_encoder",image_encoder)

input = random_tensor([1,3,4,4],1)
output = image_encoder(input)
items = [Item("input", input, "TensorFloat"), Item("output", output, "TensorFloat")]
output_to_file("image_encoder",items)

del image_encoder
del input
del output

## Transformer
#### Attention

In [ ]:
from segment_anything.modeling.transformer import Attention

attention = Attention(32,8,1)
input_to_file("transformer_attention",attention)

#Forward
q = random_tensor([1,32,32],1)
k = random_tensor([1,32,32],2)
v = random_tensor([1,32,32],3)
output = attention.forward(q,k,v)
items = [Item("q", q, "TensorFloat"), Item("k", k, "TensorFloat"), Item("v", v, "TensorFloat"), Item("output", output, "TensorFloat")]
output_to_file("transformer_attention",items)


#### TwoWayAttention

In [ ]:
from segment_anything.modeling.transformer import TwoWayAttentionBlock

block = TwoWayAttentionBlock(256,8,2048,nn.ReLU,2,False)
input_to_file("transformer_two_way_attention_block",block)

#Forward
queries = random_tensor([1,256,256],1)
keys = random_tensor([1,256,256],2)
query_pe = random_tensor([1,256,256],3)
key_pe = random_tensor([1,256,256],4)
out_queries,out_keys = block(queries,keys,query_pe,key_pe)
items = [Item("queries", queries, "TensorFloat"), Item("keys", keys, "TensorFloat"), Item("query_pe", query_pe, "TensorFloat"), Item("key_pe", key_pe, "TensorFloat"), Item("out_queries", out_queries, "TensorFloat"), Item("out_keys", out_keys, "TensorFloat")]
output_to_file("transformer_two_way_attention_block",items)


#### TwoWayTransformer

In [ ]:
from segment_anything.modeling.transformer import TwoWayTransformer

transformer = TwoWayTransformer(2, 64, 4, 256, nn.ReLU, 2)
input_to_file("transformer_two_way_transformer",transformer)

# Forward
image_embedding = random_tensor([1,64,16,16],1)
image_pe = random_tensor([1,64,16,16],2)
point_embedding = random_tensor([16, 256, 64],3)
queries,keys = transformer(image_embedding,image_pe,point_embedding)
items = [Item("image_embedding", image_embedding, "TensorFloat"), Item("image_pe", image_pe, "TensorFloat"), Item("point_embedding", point_embedding, "TensorFloat"), Item("queries", queries, "TensorFloat"), Item("keys", keys, "TensorFloat")]
output_to_file("transformer_two_way_transformer",items)

## Mask decoder
#### MLP block


In [ ]:
from segment_anything.modeling.mask_decoder import MLP

mlp = MLP(256,256,256,4,False)
input_to_file("mlp",mlp)

# Forward
input = random_tensor([1,256],1)
output = mlp(input)
items = [Item("input", input, "TensorFloat"), Item("output", output, "TensorFloat")]
output_to_file("mlp",items)

#### Mask decoder

In [ ]:
from segment_anything.modeling.mask_decoder import MaskDecoder

transformer = TwoWayTransformer(2, 64, 2, 512, nn.ReLU, 2)
mask_decoder = MaskDecoder(transformer_dim=64,transformer=transformer,num_multimask_outputs=3, activation=nn.GELU,iou_head_depth=3,iou_head_hidden_dim=64)
input_to_file("mask_decoder",mask_decoder)

# Forward
image_embedding = random_tensor([1,64,16,16],1)
image_pe = random_tensor([1,64,16,16],2)
sparse_prompt_embeddings = random_tensor([16, 2, 64],3)
dense_prompt_embeddings = random_tensor([16, 64, 16, 16],4)
masks, iou_pred = mask_decoder(image_embedding,image_pe,sparse_prompt_embeddings,dense_prompt_embeddings,True)
items = [Item("image_embedding", image_embedding, "TensorFloat"), Item("image_pe", image_pe, "TensorFloat"), Item("sparse_prompt_embeddings", sparse_prompt_embeddings, "TensorFloat"), Item("dense_prompt_embeddings", dense_prompt_embeddings, "TensorFloat"), Item("masks", masks, "TensorFloat"), Item("iou_pred", iou_pred, "TensorFloat")]
output_to_file("mask_decoder",items)

In [ ]:
# Predict masks
transformer = TwoWayTransformer(2, 64, 2, 512, nn.ReLU, 2)
mask_decoder = MaskDecoder(transformer_dim=64,transformer=transformer,num_multimask_outputs=3, activation=nn.GELU,iou_head_depth=3,iou_head_hidden_dim=64)
input_to_file("mask_decoder_predict",mask_decoder)

# Predict masks
image_embedding = random_tensor([1,64,16,16],1)
image_pe = random_tensor([1,64,16,16],2)
sparse_prompt_embeddings = random_tensor([16, 2, 64],3)
dense_prompt_embeddings = random_tensor([16, 64, 16, 16],4)
masks, iou_pred = mask_decoder.predict_masks(image_embedding,image_pe,sparse_prompt_embeddings,dense_prompt_embeddings)
items = [Item("image_embedding", image_embedding, "TensorFloat"), Item("image_pe", image_pe, "TensorFloat"), Item("sparse_prompt_embeddings", sparse_prompt_embeddings, "TensorFloat"), Item("dense_prompt_embeddings", dense_prompt_embeddings, "TensorFloat"), Item("masks", masks, "TensorFloat"), Item("iou_pred", iou_pred, "TensorFloat")]
output_to_file("mask_decoder_predict",items)


## Prompt Encoder
#### Positional Embedding

In [ ]:
from segment_anything.modeling.prompt_encoder import PositionEmbeddingRandom

# _pe_encoding 
position_embedding = PositionEmbeddingRandom(128, None)
input_to_file("position_embedding_random_pe_encoding",position_embedding)

input = random_tensor([64,69,2],1)
output = position_embedding._pe_encoding(input)
items = [Item("input", input, "TensorFloat"), Item("output", output, "TensorFloat")]
output_to_file("position_embedding_random_pe_encoding",items)

In [ ]:
# Forward
position_embedding = PositionEmbeddingRandom(128, None)
input_to_file("position_embedding_random_forward",position_embedding)

input= (64,64)
output = position_embedding.forward(input)
items = [Item("input", input, "Size"), Item("output", output, "TensorFloat")]
output_to_file("position_embedding_random_forward",items)

In [ ]:
# Forward with coords
position_embedding = PositionEmbeddingRandom(128, None)
input_to_file("position_embedding_random_forward_with_coords",position_embedding)

input = random_tensor([64,2,2],1)
image_size  = (1024,1024)
output = position_embedding.forward_with_coords(input,image_size)
items = [Item("input", input, "TensorFloat"), Item("image_size", image_size, "Size"), Item("output", output, "TensorFloat")]
output_to_file("position_embedding_random_forward_with_coords",items)

#### Prompt Encoder

In [ ]:
from segment_anything.modeling.prompt_encoder import PromptEncoder

mask_in_chans =8
embed_dim =128

In [ ]:
# Embed points pad
prompt_encoder = PromptEncoder(embed_dim,(32,32),(512,512),mask_in_chans,nn.GELU)
input_to_file("prompt_encoder_embed_points_pad",prompt_encoder)

points = random_tensor([32,1,2],1)
labels = random_tensor([32,1],2)
output = prompt_encoder._embed_points(points,labels,True)
items = [Item("points", points, "TensorFloat"), Item("labels", labels, "TensorFloat"), Item("output", output, "TensorFloat")]
output_to_file("prompt_encoder_embed_points_pad",items)

In [ ]:
# Embed points no pad
prompt_encoder = PromptEncoder(embed_dim,(32,32),(512,512),mask_in_chans,nn.GELU)
input_to_file("prompt_encoder_embed_points_no_pad",prompt_encoder)

points = random_tensor([32,1,2],1)
labels = random_tensor([32,1],2)
output = prompt_encoder._embed_points(points,labels,False)
items = [Item("points", points, "TensorFloat"), Item("labels", labels, "TensorFloat"), Item("output", output, "TensorFloat")]
output_to_file("prompt_encoder_embed_points_no_pad",items)

In [ ]:
# Embed boxes
prompt_encoder = PromptEncoder(embed_dim,(32,32),(512,512),mask_in_chans,nn.GELU)
input_to_file("prompt_encoder_embed_boxes",prompt_encoder)

boxes = random_tensor([32,4],1)
output = prompt_encoder._embed_boxes(boxes)
items = [Item("boxes", boxes, "TensorFloat"), Item("output", output, "TensorFloat")]
output_to_file("prompt_encoder_embed_boxes",items)

In [ ]:
# # Embed masks

prompt_encoder = PromptEncoder(embed_dim,(32,32),(512,512),mask_in_chans,nn.GELU)
input_to_file("prompt_encoder_embed_masks",prompt_encoder)

masks = random_tensor([8,1,4,4],1)
output = prompt_encoder._embed_masks(masks)
items = [Item("masks", masks, "TensorFloat"), Item("output", output, "TensorFloat")]
output_to_file("prompt_encoder_embed_masks",items)

In [ ]:
# forward points
prompt_encoder = PromptEncoder(embed_dim,(32,32),(512,512),mask_in_chans,nn.GELU)
input_to_file("prompt_encoder_forward_points",prompt_encoder)

points = random_tensor([8,1,2],1),random_tensor([8,1],2)
boxes = None
masks = None
sparse,dense = prompt_encoder.forward(points,boxes,masks)
items = [Item("points", points[0], "TensorFloat"),Item("labels", points[1], "TensorFloat"), Item("sparse", sparse, "TensorFloat"), Item("dense", dense, "TensorFloat")]
output_to_file("prompt_encoder_forward_points",items)

In [ ]:
# forward boxes
prompt_encoder = PromptEncoder(embed_dim,(32,32),(512,512),mask_in_chans,nn.GELU)
input_to_file("prompt_encoder_forward_boxes",prompt_encoder)

points = None
boxes = random_tensor([8,4],1)
masks = None
sparse,dense = prompt_encoder.forward(points,boxes,masks)
items = [Item("boxes", boxes, "TensorFloat"), Item("sparse", sparse, "TensorFloat"), Item("dense", dense, "TensorFloat")]
output_to_file("prompt_encoder_forward_boxes",items)

## Utils
#### ResizeLongestSide

In [ ]:
from segment_anything.utils.transforms import ResizeLongestSide

# Get Preprocess shape
resize = ResizeLongestSide(64)
output = resize.get_preprocess_shape(32,32,64)
items = [Item("output", output, "Size")]
output_to_file("resize_get_preprocess_shape",items)

In [ ]:
# Apply image
resize = ResizeLongestSide(64)
input = random_tensor([120,180,3],1).mul(255).type(torch.uint8).numpy()
output = resize.apply_image(input)
items = [Item("input", input, "TensorInt"), Item("output", output, "TensorInt")]
output_to_file("resize_apply_image",items)

In [ ]:
# Apply coords
resize = ResizeLongestSide(64)

input = random_tensor([1, 2,2],1).mul(255).type(torch.int).numpy()
original_size = (1200,1800)
output = resize.apply_coords(input,original_size)
items = [Item("input", torch.from_numpy(input).type(torch.int), "TensorInt"), Item("output", torch.from_numpy(output).type(torch.float32), "TensorFloat")]
output_to_file("resize_apply_coords",items)

In [ ]:
# Apply boxes
resize = ResizeLongestSide(64)

boxes=random_tensor([1, 4],1).mul(255).type(torch.int).numpy()
original_size = (1200,1800)
output = resize.apply_boxes(boxes,original_size)
items = [Item("boxes", boxes, "TensorInt"), Item("output", output, "TensorFloat")]
output_to_file("resize_apply_boxes",items)

In [ ]:
# Apply image torch
resize  = ResizeLongestSide(64)
input = random_tensor([1, 3, 32, 32],1)
output = resize.apply_image_torch(input)
items = [Item("input", input, "TensorFloat"), Item("output", output, "TensorFloat")]
output_to_file("resize_apply_image_torch",items)

In [ ]:
# Apply coords torch
resize  = ResizeLongestSide(64)
coords = random_tensor([32,32],1).mul(255).type(torch.int)
original_size = (32,32)
output = resize.apply_coords_torch(coords,original_size)
items = [Item("coords", coords, "TensorInt"), Item("original_size", original_size, "Size"), Item("output", output, "TensorFloat")]
output_to_file("resize_apply_coords_torch",items)

In [ ]:
# Apply boxes torch
resize  = ResizeLongestSide(64)
boxes = random_tensor([32,32],1).mul(255).type(torch.int)
original_size = (32,32)
output = resize.apply_boxes_torch(boxes,original_size)
items = [Item("boxes", boxes, "TensorInt"), Item("original_size", original_size, "Size"), Item("output", output, "TensorFloat")]
output_to_file("resize_apply_boxes_torch",items)

## Build Sam

In [ ]:
from segment_anything.build_sam import build_sam_vit_h,Sam,build_sam_vit_b,build_sam_vit_l

def get_items(sam:Sam):
    items = [
        Item("mask_threshold", sam.mask_threshold, "Float"),
        Item("image_format",sam.image_format, "String"),
        Item("pixel_mean", sam.pixel_mean, "TensorFloat"),
        Item("pixel_std", sam.pixel_std, "TensorFloat"),
        Item("mask_decoder.num_mask_tokens", sam.mask_decoder.num_mask_tokens, "Int"),
        Item("prompt_encoder.embed_dim", sam.prompt_encoder.embed_dim, "Int"),
        Item("prompt_encoder.input_image_size", sam.prompt_encoder.input_image_size, "Size"),
    ]
    return items

sam_vit_h = build_sam_vit_h()
output_to_file("sam_vit_h",get_items(sam_vit_h))

sam_vit_b = build_sam_vit_b()
output_to_file("sam_vit_b",get_items(sam_vit_b))

sam_vit_l = build_sam_vit_l()
output_to_file("sam_vit_l",get_items(sam_vit_l))

del sam_vit_h, sam_vit_b, sam_vit_l

## Sam

In [ ]:
checkpoint = "sam-convert/sam_test.pth"

In [ ]:
# Forward boxes
sam = build_sam_test(checkpoint)
# input_to_file("sam_forward_boxes",sam)
batched_input = [
     {
         'image': random_tensor([3,8,8],1).mul(255).type(torch.int),
         'boxes': random_tensor([4,4],1),
         'original_size': (100,200)
     },
     {
         'image': random_tensor([3,8,8],1).mul(255).type(torch.int),
         'boxes': random_tensor([4,4],1),
         'original_size': (50,80)
     }
]
output = sam.forward(batched_input,False)
items=[]
for i,x in enumerate(output):
    masks = x['masks']
    items.append(Item("masks"+str(i), masks, "TensorBool"))

    iou_predictions = x['iou_predictions']
    items.append(Item("iou_predictions"+str(i), iou_predictions, "TensorFloat"))

    if 'low_res_logits' in x:
        low_res_masks = x['low_res_logits']
        items.append(Item("low_res_logits"+str(i), low_res_masks, "TensorFloat"))

    
output_to_file("sam_forward_boxes",items)
del sam

In [ ]:
# Forward Points
sam = build_sam_test(checkpoint)
# input_to_file("sam_forward_points",sam)
batched_input = [
     {
         'image': random_tensor([3,8,8],1).mul(255).type(torch.int),
         'original_size': (100,200),
         'point_coords': random_tensor([4,2,2],1),
         'point_labels': random_tensor([4,2],1)
     },
     {
         'image': random_tensor([3,8,8],2).mul(255).type(torch.int),
         'original_size': (50,80),
         'point_coords': random_tensor([4,2,2],2),
         'point_labels': random_tensor([4,2],2)
         
     }
]
output = sam.forward(batched_input,False)
items=[]
for i,x in enumerate(output):
    masks = x['masks']
    items.append(Item("masks"+str(i), masks, "TensorBool"))

    iou_predictions = x['iou_predictions']
    items.append(Item("iou_predictions"+str(i), iou_predictions, "TensorFloat"))

    if 'low_res_logits' in x:
        low_res_masks = x['low_res_logits']
        items.append(Item("low_res_logits"+str(i), low_res_masks, "TensorFloat"))

    
output_to_file("sam_forward_points",items)
del sam

In [ ]:
# Postprocess masks
sam = build_sam_test(checkpoint)

masks = random_tensor([4,1,256,256],1)
input_size = (684,1024)
original_size = (534,800)
output = sam.postprocess_masks(masks,input_size,original_size)
items = [Item("masks", masks, "TensorFloat"), Item("input_size", input_size, "Size"), Item("original_size", original_size, "Size"), Item("output", output, "TensorFloat")]
output_to_file("sam_postprocess_masks",items)
del sam

In [ ]:
# Preprocess
sam = build_sam_test(checkpoint)

input = random_tensor([3,171,128],1).mul(255).type(torch.int)
output = sam.preprocess(input)
items = [Item("input", input, "TensorInt"), Item("output", output, "TensorFloat")]
output_to_file("sam_preprocess",items)
del sam

## Sam Predictor

In [ ]:
# Init
from segment_anything.predictor import SamPredictor

def get_predictor(with_set_image:bool=False):
    sam = build_sam_test(checkpoint)
    predictor = SamPredictor(sam)
    if with_set_image:
        image = random_tensor([120,180,3],1).mul(255).type(torch.uint8).numpy()
        predictor.set_image(image,"RGB")
    return predictor


In [ ]:
# Set image 
predictor = get_predictor(True)

items=[
    Item("original_size",predictor.original_size,"Size"),
    Item("input_size",predictor.input_size,"Size"),
    Item("features",predictor.features,"TensorFloat"),
    Item("is_image_set",predictor.is_image_set,"Bool"),
]
output_to_file("predictor_set_image",items)
del predictor,

In [ ]:
# Set torch image
predictor = get_predictor()

image = random_tensor([1, 3, 683, 1024],1).mul(255).type(torch.uint8)
original_size = (120, 180)
predictor.set_torch_image(image,original_size)
items=[
    Item("original_size",predictor.original_size,"Size"),
    Item("input_size",predictor.input_size,"Size"),
    Item("features",predictor.features,"TensorFloat"),
    Item("is_image_set",predictor.is_image_set,"Bool"),
]
output_to_file("predictor_set_torch_image",items)

In [ ]:
# Predict
predictor = get_predictor(True)

point_coords = random_ndarray([1,2],1)
point_labels = random_tensor([1],1).type(torch.int).numpy()

masks, iou_predictions, low_res_masks =predictor.predict(point_coords,point_labels,None,None,True,False)
items =[
    Item("masks", masks, "TensorBool"),
    Item("iou_predictions", iou_predictions, "TensorFloat"),
    Item("low_res_masks", low_res_masks, "TensorFloat"),
]
output_to_file("predictor_predict",items)
del point_coords, point_labels, masks, iou_predictions, low_res_masks,predictor

In [ ]:
# Predict torch
predictor = get_predictor(True)

point_coords = random_tensor([1,1,2],1)
point_labels = random_tensor([1,1],1)
masks, iou_predictions, low_res_masks = predictor.predict_torch(point_coords,point_labels,None,None,True,False)
items =[
    Item("masks", masks, "TensorBool"),
    Item("iou_predictions",iou_predictions, "TensorFloat"),
    Item("low_res_masks", low_res_masks, "TensorFloat")
]
output_to_file("predictor_predict_torch",items)
del predictor,point_coords,point_labels,masks,iou_predictions,low_res_masks